https://jira.x5.ru/browse/CVMUPG-797

In [1]:
name = 'blago_base_conversion'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='cvm5-rnd', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

# seg1

### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [10]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .limit(1000000)
                     .select('account_no', 'customer_rk')
                            )

### По этой активной базе отсекаем ГКГ, проверяем доступность в канале смс вайбер на 25 ноября.

In [11]:
check_date = '2021-12-25'

In [12]:
guests = remove_global_control_group(spark, customers_usecase, check_date)

 Фильтр Глобальная контрольная группа - валиден.


[Stage 7:======================================================>(297 + 3) / 300]

	 Отсёк 85 862 записей.


In [13]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [16]:
exclude_camp_wave_id = [73554, 73555]

In [18]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         exclude_camp_wave_id=exclude_camp_wave_id,
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [19]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [20]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [21]:
seg_sms.write.parquet('temp_blago1', mode='overwrite')
seg_sms = spark.read.parquet('temp_blago1')

In [22]:
seg_sms.count()

521641

### Собираем чеки этих гостей

In [23]:
start_date = datetime.date(2021, 11, 24)
end_date = datetime.date(2021, 11, 30)

In [24]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )


In [25]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
#                         .filter(((F.col('loyalty_cardholder_age_yrs') >= 25) & 
#                                 (F.col('loyalty_cardholder_age_yrs') <= 65)) | 
#                                 (~F.col('loyalty_cardholder_age_yrs').isNotNull()))
#                         .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [26]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [27]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


#### Отбираем тех, кто покупал продукты в категории масло

##### Отбираем PLU определенной категории (масло растительное) 1 л

In [29]:
plu_hierarchy_lvl_4_dk = ['FD0212003']
# syntethic_category_id = [51,26,52]
plu_brand_code = ['5256']
# plu_not_in = [4138521]

In [30]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin([74]))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [31]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [32]:
checks_tc5 = checks_items.join(checks_headers_tc5, on='rtl_txn_id', how='inner')

In [33]:
seg = (checks_tc5
           .select('customer_rk')
           .distinct()
      )

In [34]:
seg.count()

4026

In [35]:
4026/521641*100

0.7717951618066831

In [36]:
seg.write.parquet('temp_blago2', mode='overwrite')
seg = spark.read.parquet('temp_blago2')

In [37]:
seg.count()

4026

In [ ]:
seg_pd = seg.toPandas()

In [ ]:
cross, upgrade, ump = get_usecases(seg_pd)

In [ ]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [ ]:
print('seg_no: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_no, len(cross), len(upgrade), len(ump)))